# Load library and dataset

In [1]:
import warnings

warnings.filterwarnings("ignore")
import torch
import numpy as np
import pandas as pd
import os
import itertools
import time

from tqdm import tqdm_notebook as tqdm
from torch import optim
from torch.utils.data import TensorDataset, DataLoader
from model.model import sim_trip_classification
from model.SelfAttentionModel import StructuredSelfAttention
from utils.data_loader import (
    load_data_set,
    load_word_to_index,
    load_char_to_index,
    load_triplet_orders,
    load_padded_data,
    load_triplet,
    generate_embedding,
)
from utils.pretrained_glove_embeddings import load_glove_embeddings
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.utils import shuffle
from sklearn.metrics import f1_score, accuracy_score, recall_score, precision_score

In [2]:
# main_path = "/data/dac/dedupe-project/openmap/"  # Open map dataset
main_path = "/data/dac/dedupe-project/new/"  # Lab dataset

In [3]:
def prepare_data(file_path, retrain=True):
    # Load dataset
    df = load_data_set(file_path)
    df.fillna("", inplace=True)
    df.reset_index(inplace=True, drop=True)
    # get char to index and embedding whole dataset
    embedding_index = load_char_to_index(df)
    embeddings = generate_embedding(embedding_index, embedding_dim=50)
    # Get X and X_len as matrix
    X, X_len = load_padded_data(df, embedding_index, char_level=True)

    def truncate_non_string(X, X_len):
        # Drop rows that have length of word vector = 0
        truncate_index = [i for i in range(0, len(X_len)) if X_len[i] <= 0]
        X, X_len = (
            np.delete(X, truncate_index, axis=0),
            np.delete(X_len, truncate_index, axis=0),
        )

        return X, X_len, sorted(truncate_index, reverse=True)

    X, X_len, truncate_index = truncate_non_string(X, X_len)
    df.drop(index=truncate_index, inplace=True)
    df.reset_index(inplace=True, drop=True)

    return df, X, X_len, embeddings, embedding_index


def to_cuda(loader, device):
    return [load.to(device) for load in loader]

In [4]:
# set cuda device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# torch.cuda.set_device(device)

# random_augment_train.csv
# "new_generated_labeled_data.csv"
full_generated_data_path = "random_augment_train.csv"  # Local dataset
open_map_data_path = "openmap-us-train.csv"  # Open map dataset
# Remember to set open map in train data
df, X, X_len, embeddings, embedding_index = prepare_data(
    main_path + full_generated_data_path
)

Load character to index successfully!



Load padded data successfully!


In [5]:
# import matplotlib.pyplot as plt

# plt.hist(X_len)
# plt.savefig("/data/dac/dedupe-project/image/length_X.eps", format="eps", dpi=1200)

# Building Model

## Generate DataLoader and Triplet orders

In [6]:
batch_size = 10000
df_triplet_orders = load_triplet_orders(df)
print("Loading triplet order successfully!")
anc_loader, pos_loader, neg_loader = load_triplet(
    np.array(X), X_len, df_triplet_orders, batch_size=batch_size
)
print("Load triplet data successfully!")


Loading triplet order successfully!



Load triplet data successfully!


In [7]:
# # Create train file as pair for some methods
# train_pair_arr = []
# for row in df_triplet_orders.loc[:20000, :].itertuples():
#     anchor = row[2]
#     pos = row[3]
#     neg = row[4]

#     pair = {}
#     pair["address"] = df.iloc[anchor].content
#     pair["duplicated_address"] = df.iloc[pos].content
#     pair["similar"] = 1
#     train_pair_arr.append(pair)

#     pair = {}
#     pair["address"] = df.iloc[anchor].content
#     pair["duplicated_address"] = df.iloc[neg].content
#     pair["similar"] = 0
#     train_pair_arr.append(pair)

# pd.DataFrame(train_pair_arr).to_csv(
#     "/data/dac/dedupe-project/train_as_pair.csv", encoding="utf-8"
# )

## Prepare validate dataset
1. Load dataset
2. Pre-processing
3. Create pair of data

In [8]:
path = "/data/dac/dedupe-project/test/"
test_df = pd.read_csv(path + "test_address_3.csv", encoding="ISO-8859-1")
test_df.fillna("", inplace=True)
test_df.reset_index(inplace=True)
test_df_1 = test_df.loc[:, ["address"]]
test_df_1["content"] = (
    test_df_1["address"]
    .str.lower()
    .str.replace("\n", " ")
    .str.replace(r"[ ]+", " ", regex=True)
    .str.replace("null", "")
    .str.replace("nan", "")
)
test_df_2 = test_df.loc[:, ["duplicated_address"]]
test_df_2["content"] = (
    test_df_2["duplicated_address"]
    .str.lower()
    .str.replace("\n", " ")
    .str.replace(r"[ ]+", " ", regex=True)
    .str.replace("null", "")
    .str.replace("nan", "")
)

In [9]:
def data_loader(test_df_1, test_df_2):
    # Data Preparation pipeline
    # Create Dataloader based on two dataframe have row 'content' in it
    X1, X1_lens = load_padded_data(pd.DataFrame(test_df_1), embedding_index)

    X2, X2_lens = load_padded_data(pd.DataFrame(test_df_2), embedding_index)

    # Drop rows that have length of word vector = 0
    truncate_index = [
        i for i in range(0, len(X1_lens)) if (X1_lens[i] <= 0 or X2_lens[i] <= 0)
    ]
    X1, X1_lens = (
        np.delete(X1, truncate_index, axis=0),
        np.delete(X1_lens, truncate_index, axis=0),
    )
    X2, X2_lens = (
        np.delete(X2, truncate_index, axis=0),
        np.delete(X2_lens, truncate_index, axis=0),
    )

    def create_data_loader(X, batch_size=batch_size):
        X, X_lens = np.array(X[0]), np.array(X[1])

        # Create data loader
        data = TensorDataset(
            torch.from_numpy(X).type(torch.LongTensor), torch.ByteTensor(X_lens)
        )
        loader = DataLoader(data, batch_size=batch_size, drop_last=False)
        return loader

    return (
        create_data_loader([X1, X1_lens]),
        create_data_loader([X2, X2_lens]),
        truncate_index,
    )


def create_test(n, test_df_1, test_df_2):
    # Generate small test based on ground truth
    test_df_1a = pd.DataFrame()
    test_df_1b = pd.DataFrame()

    for i1, i2 in shuffle(list(itertools.combinations(test_df_1.index, 2)))[:n]:
        try:
            test_df_1a = test_df_1a.append(test_df_1.iloc[i1, :])
            test_df_1b = test_df_1b.append(test_df_2.iloc[i2, :])
        except:
            print(i1, i2)

    test_df_1b = test_df_1b.append(test_df_1)
    test_df_1a = test_df_1a.append(test_df_2)

    test_df_1a.reset_index(inplace=True)
    test_df_1b.reset_index(inplace=True)

    return test_df_1a, test_df_1b


def validate(model, X1, X2, device):
    y_true = []
    y_pred = []
    for a, b in tqdm(zip(X1, X2)):
        # Send data to graphic card - Cuda0
        a, b = to_cuda(a, device), to_cuda(b, device)
        with torch.no_grad():
            # Predict
            y_pred_curr = model(a, b).data.numpy()
            y_pred_curr[y_pred_curr <= 0] = 0
            y_pred_curr[y_pred_curr > 0] = 1
            y_pred = np.concatenate([y_pred, y_pred_curr])
            # Get the truth label
            y_true_curr = np.ones(len(dist))
            y_true = np.concatenate([y_true, y_true_curr])
    print(
        "Accuracy:\t{}\t\tF1-score:\t{}\t".format(
            round(accuracy_score(y_true, y_pred), 4), round(f1_score(y_true, y_pred), 4)
        ),
        end="",
    )
    print(
        "Precision:\t{}\t\tRecall:\t{}".format(
            round(precision_score(y_true, y_pred), 4),
            round(recall_score(y_true, y_pred), 4),
        ),
        end="",
    )

In [10]:
test_df_1a, test_df_1b = create_test(1176, test_df_1, test_df_2)
X1, X2, drop = data_loader(test_df_1a, test_df_1b)


Load padded data successfully!



Load padded data successfully!


## Triplet Siamese
Build, train and validate Triplet Siamese Model
1. Embedding
2. Deep Neural Network (Loss: Minimize the loss by maximize the distance between positive and negative)
3. Calculate Distance

In [11]:
# Self-attention triplet model
# model = StructuredSelfAttention(embeddings=embeddings, n_classes=50).to(device)

# triplet_siamese_300d_bi_gru: hit
# triplet_siamese_50d_bi_gru_random: outperform
triplet_model_path = "/data/dac/dedupe-project/new/model/stc_l1_h50_m7_n30"

lr = 0.1
margin = 0.4
# Load model & optimizer
model = sim_trip_classification(
    embeddings=embeddings, layers=1, hid_dim=50, max_len=7, n_classes=30
).to(device)
optimizer = optim.Adam(model.parameters(), lr=lr)
print(model)

sim_trip_classification(
  (embeddings): Embedding(59, 50)
  (gru): GRU(50, 50, batch_first=True, dropout=0.2, bidirectional=True)
  (linear_final): Linear(in_features=100, out_features=30, bias=True)
  (gru_combined): GRU(2, 5, batch_first=True, dropout=0.2, bidirectional=True)
  (linear_class): Linear(in_features=30, out_features=1, bias=True)
)


In [ ]:
# Load model and optimizer
checkpoint = torch.load(triplet_model_path, map_location=device)
model.load_state_dict(checkpoint["model"])
optimizer.load_state_dict(checkpoint["optimizer"])

model.eval()

In [13]:
# Train model
epochs = 2
best_lost = None
early_stopping_steps = 5

loss_list = []
average_list = []
model.train()

start_time = time.time()
for epoch in tqdm(range(epochs), desc="Epoch", total=epochs):
    avg_loss = 0
    avg_acc = 0

    for batch, [anc_x, pos_x, neg_x] in enumerate(
        zip(anc_loader, pos_loader, neg_loader)
    ):
        # Training model per batch
        # Send data to graphic card - Cuda
        anc_x, pos_x, neg_x = (
            to_cuda(anc_x, device),
            to_cuda(pos_x, device),
            to_cuda(neg_x, device),
        )
        loss = model(anc_x, pos_x, neg_x)

        # Append to batch list
        avg_loss += float(loss)

        # Gradient
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        print(
            "\rBatch:\t{}\tLoss:\t{}".format(batch, round(float(loss), 4)), end="",
        )
        validate(model, X1, X2, device)
    # Average loss all epochs
    avg_loss /= len(anc_loader)

    loss_list.append(avg_loss)
    print(
        "\rEpoch:\t{}\tAverage Loss:\t{}".format(
            epoch,
            round(avg_loss, 4)
        )
    )
    if best_lost is None or best_lost > avg_loss:
        best_lost = avg_loss
        forward_index = 0
        #         Save model
        torch.save(
            {"model": model.state_dict(), "optimizer": optimizer.state_dict()},
            triplet_model_path,
        )
    else:
        # Early stopping after reachs {early_stopping_steps} steps
        forward_index += 1
        if forward_index == early_stopping_steps or best_lost == 0:
            break
    if best_lost < 1:
        break
print("--- %s seconds ---" % (time.time() - start_time))

AttributeError: 'Tensor' object has no attribute 'batch_sizes'

In [ ]:
# Save model
torch.save(
    {"model": model.state_dict(), "optimizer": optimizer.state_dict()},
    triplet_model_path,
)

# Test

## Fodor

In [17]:
def create_pairs(df1, df2, pair):
    df_out1 = []
    df_out2 = []
    y = []
    for row in pair.itertuples():
        # Create two test dataframe based on index file
        # index, l_id, r_id, label
        df_out1.append(df1["content"][df1["id"] == row[1]])
        df_out2.append(df2["content"][df2["id"] == row[2]])
        y.append(row[3])

    df_out1 = pd.DataFrame(pd.concat(df_out1))
    df_out2 = pd.DataFrame(pd.concat(df_out2))

    # Create DataLoader
    X1, X2, drop = data_loader(df_out1, df_out2)

    return X1, X2, y, df_out1, df_out2

In [138]:
df_train_1

,content
6,6703 melrose ave. los angeles californian
22,67 n. raymond ave. los angeles asian
501,1915 fillmore st. san francisco italian
38,160 e. 64th st. new york american
70,3570 las vegas blvd. s las vegas continental
...,...
108,3201 fillmore st. san francisco mediterranean
133,730 n. la cienega blvd. los angeles french
86,224 ponce de leon ave. atlantasouthern
424,3645 las vegas blvd. s las vegas buffets


In [139]:
df_train_2

,content
230,8284 melrose ave. los angeles french bistro
240,67 n. raymond ave. pasadenachinese
214,1870 fillmore st. san francisco american
256,160 e. 64th st. new york city french bistro
130,3595 las vegas blvd. s. las vegas continental
...,...
326,3127 fillmore st. san francisco american ...
229,903 n. la cienega blvd. w. hollywood fren...
156,600 ponce de leon ave. atlantaitalian
125,3799 las vegas blvd. s. las vegas seafood


In [20]:
# Prepare data to input as DataLoader
fd_a = pd.read_csv(path + "fodor/tableA.csv")
fd_a["content"] = fd_a["addr"] + fd_a["city"] + fd_a["type"]
fd_a["content"] = fd_a["content"].str.replace("'", "").str.replace("`", " ")

fd_b = pd.read_csv(path + "fodor/tableB.csv")
fd_b["content"] = fd_b["addr"] + fd_b["city"] + fd_b["type"]
fd_b["content"] = fd_b["content"].str.replace("'", "").str.replace("`", " ")

fd_test_pair = pd.read_csv(path + "fodor/test.csv")
fd_train_pair = pd.read_csv(path + "fodor/train.csv")

X1_test, X2_test, y_test, df_test_1, df_test_2 = create_pairs(fd_a, fd_b, fd_test_pair)
X1_train, X2_train, y_train, df_train_1, df_train_2 = create_pairs(
    fd_a, fd_b, fd_train_pair
)


Load padded data successfully!



Load padded data successfully!



Load padded data successfully!



Load padded data successfully!


In [23]:
fd_a

,id,name,addr,city,phone,type,class,content
0,0,` arnie morton \ 's of chicago ',' 435 s. la cienega blv . ',` los angeles ',310/246 -1501,american,0,435 s. la cienega blv . los angeles american
1,1,` art \ 's delicatessen ',' 12224 ventura blvd. ',` studio city ',818/762 -1221,american,1,12224 ventura blvd. studio city american
2,2,` hotel bel-air ',' 701 stone canyon rd. ',` bel air ',310/472 -1211,californian,2,701 stone canyon rd. bel air californian
3,3,` cafe bizou ',' 14016 ventura blvd. ',` sherman oaks ',818/788 -3536,french,3,14016 ventura blvd. sherman oaks french
4,4,campanile,' 624 s. la brea ave. ',` los angeles ',213/938 -1447,american,4,624 s. la brea ave. los angeles american
...,...,...,...,...,...,...,...,...
528,528,` yank sing ',' 427 battery st. ',` san francisco ',415/541 -4949,asian,528,427 battery st. san francisco asian
529,529,` yaya cuisine ',' 1220 9th ave. ',` san francisco ',415/566 -6966,` greek and middle eastern ',529,1220 9th ave. san francisco greek and mid...
530,530,` yoyo tsumami bistro ',' 1611 post st. ',` san francisco ',415/922 -7788,french,530,1611 post st. san francisco french
531,531,zarzuela,' 2000 hyde st. ',` san francisco ',415/346 -0800,` mexican/latin american/spanish ',531,2000 hyde st. san francisco mexican/latin...


In [128]:
y_pred = []
pred_list = []
for a, b in tqdm(zip(X1, X2)):
    # Send data to graphic card - Cuda0
    a, b = to_cuda(a, device), to_cuda(b, device)
    with torch.no_grad():
        a, b = model(a, b)
        a, b = a.cpu(), b.cpu()
        a = a.reshape(a.shape[0], -1)
        b = b.reshape(b.shape[0], -1)
        #         att1 = att1.cpu()
        #         att2 = att2.cpu()
        dist = np.array(
            [
                cosine_similarity([a[i].numpy()], [b[i].numpy()])
                for i in range(0, len(a))
            ]
        ).flatten()
        y_pred_curr = np.ones(len(dist))
        y_pred_curr[np.where(dist < 0.35039073)[0]] = 0
        y_pred = np.concatenate([y_pred, y_pred_curr])

        pred_list = np.concatenate([pred_list, dist])

print(
    "Accuracy:\t{}\t\tF1-score:\t{}\t".format(
        round(accuracy_score(y_test, y_pred), 4), round(f1_score(y_test, y_pred), 4)
    ),
    end="",
)
print(
    "Precision:\t{}\t\tRecall:\t{}".format(
        round(precision_score(y_test, y_pred), 4),
        round(recall_score(y_test, y_pred), 4),
    ),
    end="",
)


Accuracy:	0.2222		F1-score:	0.2304	Precision:	0.1302		Recall:	1.0

In [21]:
df_test_1[np.array(y_test) == 0]

,content
6,6703 melrose ave. los angeles californian
501,1915 fillmore st. san francisco italian
70,3570 las vegas blvd. s las vegas continental
384,260 e. paces ferry road atlantacontinental
76,3130 piedmont road atlantaamerican
...,...
422,3393 peachtree rd. atlantacontinental
133,730 n. la cienega blvd. los angeles french
86,224 ponce de leon ave. atlantasouthern
424,3645 las vegas blvd. s las vegas buffets


In [22]:
df_test_2[np.array(y_test) == 0]

,content
230,8284 melrose ave. los angeles french bistro
214,1870 fillmore st. san francisco american
130,3595 las vegas blvd. s. las vegas continental
300,490 e. paces ferry rd. ne atlantacontinental
26,1023 abbot kinney blvd. venice american ( ne...
...,...
292,2355 peachtree rd. ne atlantaitalian
229,903 n. la cienega blvd. w. hollywood fren...
156,600 ponce de leon ave. atlantaitalian
125,3799 las vegas blvd. s. las vegas seafood


## Test 2

In [ ]:
test_df_new = pd.read_csv(path + "test.csv")
# drop_single = [i for i in test_df_new.ID.unique() if sum(test_df_new.ID == i) <2]
# test_df_new.set_index('ID', inplace=True)
# test_df_new.drop(drop_single, inplace=True)

# # Generate pairs
# pairs = []
# for i in test_df_new.index.unique():
#     temp = test_df_new[test_df_new.index == i]
#     # Generate a pair on the same row and append them to pairs
#     pair = [temp.iloc[0, 1], temp.iloc[1, 1]]
#     pairs.append(pair)

# test_df_new = pd.DataFrame(pairs, columns=['address', 'duplicated_address'])

test_df_new.fillna("", inplace=True)
test_df_new.reset_index(inplace=True)
test_df_1 = test_df_new.loc[:, ["address"]]
test_df_1["content"] = (
    test_df_1["address"]
    .str.lower()
    .str.replace("\n", " ")
    .str.replace(r"[ ]+", " ", regex=True)
    .str.replace("null", "")
    .str.replace("nan", "")
)
test_df_2 = test_df_new.loc[:, ["duplicated_address"]]
test_df_2["content"] = (
    test_df_2["duplicated_address"]
    .str.lower()
    .str.replace("\n", " ")
    .str.replace(r"[ ]+", " ", regex=True)
    .str.replace("null", "")
    .str.replace("nan", "")
)

In [ ]:
def test(n, df1, df2):
    total_acc = 0
    total_f1 = 0
    for i in range(0, 1):
        test_df_1a, test_df_1b = create_test(n, df1, df2)
        X1, X2, drop = data_loader(test_df_1a, test_df_1b)

        pred_list = np.array([])
        y_true = np.array([])
        y_pred = np.array([])
        # att1_list = []
        # att2_list = []
        for a, b in tqdm(zip(X1, X2)):
            # Send data to graphic card - Cuda0
            a, b = to_cuda(a, device), to_cuda(b, device)
            with torch.no_grad():
                a, b = model(a, b)
                a, b = a.cpu(), b.cpu()
                a = a.reshape(a.shape[0], -1)
                b = b.reshape(b.shape[0], -1)
                #         att1 = att1.cpu()
                #         att2 = att2.cpu()
                dist = np.array(
                    [
                        cosine_similarity([a[i].numpy()], [b[i].numpy()])
                        for i in range(0, len(a))
                    ]
                ).flatten()

                y_true_curr = np.zeros(len(dist))
                y_true = np.concatenate([y_true, y_true_curr])
                if len(y_true) >= n:
                    y_true[n:] = 1

                y_pred_curr = np.ones(len(dist))
                y_pred_curr[np.where(dist <= 0.72)[0]] = 0
                y_pred = np.concatenate([y_pred, y_pred_curr])

                pred_list = np.concatenate([pred_list, dist])
        #         att1_list.append(att1)
        #         att2_list.append(att2)
        total_acc += accuracy_score(y_true, y_pred)
        total_f1 += f1_score(y_true, y_pred)
        print(
            "Accuracy:\t{}\t\tF1-score:\t{}\t\t".format(
                round(accuracy_score(y_true, y_pred), 4),
                round(f1_score(y_true, y_pred), 4),
            ),
            end="",
        )
        print(
            "Precision:\t{}\t\tRecall:\t{}\t\t".format(
                round(precision_score(y_true, y_pred), 4),
                round(recall_score(y_true, y_pred), 4),
            ),
            end="",
        )

    print(
        "\nAccuracy:\t{}\t\tF1-score:\t{}\t\t".format(
            round(total_acc, 4), round(total_f1, 4)
        ),
        end="",
    )
    return test_df_1a, test_df_1b


test_df_1a, test_df_1b = test(5000, test_df_1, test_df_2)

# Mistake

In [ ]:
test_df = pd.read_csv(
    "/data/dac/dedupe-project/test/new/GT_added.csv", encoding="ISO-8859-1"
)
mistakes_df = pd.read_excel("/data/dac/dedupe-project/test/new/mistakes.xlsx")

In [ ]:
mistakes = [
    "misunderstanding",
    "typing error (translate)",
    "don’t know zipcode",
]
mistake_dict = {
    mistake: list(mistakes_df[mistake].dropna().index) for mistake in mistakes
}

In [ ]:
test_df.fillna("", inplace=True)
test_df.reset_index(inplace=True)
test_df_1 = test_df.loc[:, ["address"]]
test_df_1["content"] = (
    test_df_1["address"]
    .str.lower()
    .str.replace("\n", " ")
    .str.replace(r"[ ]+", " ", regex=True)
)
test_df_2 = test_df.loc[:, ["duplicated_address"]]
test_df_2["content"] = (
    test_df_2["duplicated_address"]
    .str.lower()
    .str.replace("\n", " ")
    .str.replace(r"[ ]+", " ", regex=True)
)

In [ ]:
X1, X2, truncate = data_loader(test_df_2, test_df_1)
test_df_1.drop(truncate, inplace=True)
# test_df_1.reset_index(inplace=True)
test_df_2.drop(truncate, inplace=True)
# test_df_2.reset_index(inplace=True)

pred_list = np.array([])
y_true = np.array([])
y_pred = np.array([])
att1_list = []
att2_list = []
for a, b in tqdm(zip(X1, X2)):
    # Send data to graphic card - Cuda0
    a, b = to_cuda(a), to_cuda(b)
    with torch.no_grad():
        a, b = model(a, b)
        a, b = a.cpu(), b.cpu()
        a = a.reshape(a.shape[0], -1)
        b = b.reshape(b.shape[0], -1)
        #         att1 = att1.cpu()
        #         att2 = att2.cpu()
        dist = np.array(
            [
                cosine_similarity([a[i].numpy()], [b[i].numpy()])
                for i in range(0, len(a))
            ]
        ).flatten()

        y_true_curr = np.ones(len(dist))
        y_true = np.concatenate([y_true, y_true_curr])

        y_pred_curr = np.ones(len(dist))
        y_pred_curr[np.where(dist <= 0.83)[0]] = 0
        y_pred = np.concatenate([y_pred, y_pred_curr])

        pred_list = np.concatenate([pred_list, dist])
#         att1_list.append(att1)
#         att2_list.append(att2)

print(
    "Accuracy:\t{}\t\tF1-score:\t{}\t\t".format(
        round(accuracy_score(y_true, y_pred), 4), round(f1_score(y_true, y_pred), 4)
    ),
    end="",
)
print(
    "Precision:\t{}\t\tRecall:\t{}\t\t".format(
        round(precision_score(y_true, y_pred), 4),
        round(recall_score(y_true, y_pred), 4),
    ),
    end="",
)

In [ ]:
for key, value in mistake_dict.items():
    y_t = y_true[value]
    y_p = y_pred[value]
    print(key)
    print(
        "Accuracy:\t{}\t\tF1-score:\t{}\t\t".format(
            round(accuracy_score(y_t, y_p), 4), round(f1_score(y_t, y_p), 4)
        ),
        end="",
    )
    print(
        "Precision:\t{}\t\tRecall:\t{}\t\t".format(
            round(precision_score(y_t, y_p), 4), round(recall_score(y_t, y_p), 4),
        )
    )

In [ ]:
test_df_1.to_csv("/data/dac/dedupe-project/test/new/GT_added.csv", encoding="utf-8")